In [1]:
import os
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from scipy.special import logsumexp
from collections import OrderedDict
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

import wandb

import warnings
warnings.filterwarnings('ignore')

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

SEED = 42
seed_everything(SEED)

In [3]:
def init_wandb(project_name="geology-forecast-challenge-sweep-gpu-bayes-30-server-HybridCNNLSTM", config=None):
    # Если run уже существует, просто возвращаем его
    if wandb.run is not None:
        return wandb.run
    
    try:
        wandb_api_key = os.environ['WANDB_API_KEY']
        
        wandb.login(key=wandb_api_key)
        
        run = wandb.init(
            project=project_name,
            config=config,
            tags=["LSTM", "Geology Forecast Challenge", "Feature Engineering"],
            reinit=True
        )
        
        print("W&B successfully initialized")
        return run
    
    except Exception as e:
        print(f"Error initializing W&B: {str(e)}")
        return None

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
train = pd.read_csv("data/train.csv").fillna(0)
test = pd.read_csv("data/test.csv").fillna(0)
sub = pd.read_csv('data/sample_submission.csv')

In [6]:
FEATURES = [c for c in test.columns if c != 'geology_id']
TARGETS = [c for c in sub.columns if c != 'geology_id']
solution = train[['geology_id'] + TARGETS].copy()
train_sub = train[['geology_id'] + TARGETS].copy()

In [7]:
def engineer_features(data, is_test=False):
    feature_data = pd.DataFrame({'geology_id': data['geology_id']})
    
    historical_cols = [col for col in data.columns if col != 'geology_id' and col.startswith('-') or col == '0']
    
    historical_cols.sort(key=lambda x: int(x) if x.isdigit() else int(x))
    
    historical_data = data[historical_cols].values
    
    # 1. Calculate local slopes (first derivative)
    slopes = np.zeros_like(historical_data)
    for i in range(1, historical_data.shape[1]):
        slopes[:, i] = historical_data[:, i] - historical_data[:, i-1]
    
    # 2. Calculate curvature (second derivative)
    curvature = np.zeros_like(historical_data)
    for i in range(1, slopes.shape[1]-1):
        curvature[:, i] = slopes[:, i+1] - slopes[:, i]
    
    # 3. Create rolling statistics for last N points
    window_sizes = [5, 10, 20]
    for window in window_sizes:

        if historical_data.shape[1] >= window:
            
            feature_data[f'mean_last_{window}'] = np.mean(historical_data[:, -window:], axis=1)
            feature_data[f'std_last_{window}'] = np.std(historical_data[:, -window:], axis=1)

            x = np.arange(window)
            
            for i in range(historical_data.shape[0]):
                
                y = historical_data[i, -window:]
                
                if np.all(y == 0):
                    feature_data.loc[i, f'trend_last_{window}'] = 0
                else:
                    slope = np.polyfit(x, y, 1)[0]
                    feature_data.loc[i, f'trend_last_{window}'] = slope
    
    # 4. Calculate smoothed versions of data (different levels of smoothing)
    smooth_windows = [3, 5, 9]
    for window in smooth_windows:
        # Savitzky-Golay filter requires window_length > polyorder
        if window <= 3: 
            continue
        
        if historical_data.shape[1] >= window:
            sg_window = window if window % 2 == 1 else window + 1
            
            for i in range(historical_data.shape[0]):
                data_slice = historical_data[i, -50:]
                if len(data_slice) >= sg_window:
                    try:
                        polyorder = 2 if sg_window <= 5 else 3
                        smoothed = savgol_filter(data_slice, sg_window, polyorder, mode='nearest')
                        feature_data.loc[i, f'sg_smooth_{window}'] = smoothed[-1]
                        
                        if len(smoothed) >= 3:
                            feature_data.loc[i, f'sg_smooth_slope_{window}'] = smoothed[-1] - smoothed[-2]
                    except Exception as e:
                        feature_data.loc[i, f'sg_smooth_{window}'] = data_slice[-1]
                        if len(data_slice) >= 3:
                            feature_data.loc[i, f'sg_smooth_slope_{window}'] = data_slice[-1] - data_slice[-2]
                else:
                    feature_data.loc[i, f'sg_smooth_{window}'] = historical_data[i, -1] if historical_data.shape[1] > 0 else 0
                    feature_data.loc[i, f'sg_smooth_slope_{window}'] = 0
    
    # 5. Calculate frequency-domain features (FFT-based)
    if historical_data.shape[1] >= 32: 
        for i in range(historical_data.shape[0]):
            fft_vals = np.abs(np.fft.rfft(historical_data[i, -32:]))
            feature_data.loc[i, 'dominant_freq'] = np.argmax(fft_vals[1:]) + 1 if len(fft_vals) > 1 else 0
            feature_data.loc[i, 'dominant_power'] = np.max(fft_vals[1:]) if len(fft_vals) > 1 else 0
            feature_data.loc[i, 'total_power'] = np.sum(fft_vals[1:]) if len(fft_vals) > 1 else 0
    
    # 6. Detect potential fault indicators
    if historical_data.shape[1] >= 5:
        max_changes = []
        for i in range(historical_data.shape[0]):
            max_change = 0
            for j in range(historical_data.shape[1] - 5):
                change = np.max(historical_data[i, j:j+5]) - np.min(historical_data[i, j:j+5])
                max_change = max(max_change, change)
            max_changes.append(max_change)
        feature_data['max_change_5pt'] = max_changes
    
    # 7. Calculate geological dip angle features
    if historical_data.shape[1] >= 10:
        dips = []
        for i in range(historical_data.shape[0]):
            # Use linear regression to find dip angle
            x = np.arange(10)
            y = historical_data[i, -10:]
            slope = np.polyfit(x, y, 1)[0]
            # Convert to degrees (slope is rise/run, arctangent gives angle)
            dip_angle = np.degrees(np.arctan(slope))
            dips.append(dip_angle)
        feature_data['dip_angle'] = dips
    
    # 8. Add the raw historical data (last 50 points)
    for i in range(min(50, historical_data.shape[1])):
        feature_data[f'raw_{i}'] = historical_data[:, -(i+1)]
    
    # 9. Create interaction features from important raw features
    if 'mean_last_5' in feature_data.columns and 'trend_last_10' in feature_data.columns:
        feature_data['mean_trend_interaction'] = feature_data['mean_last_5'] * feature_data['trend_last_10']
    
    if 'dip_angle' in feature_data.columns and 'max_change_5pt' in feature_data.columns:
        feature_data['dip_change_interaction'] = feature_data['dip_angle'] * feature_data['max_change_5pt']

    # 10. Non-linear transformations of important features
    for col in feature_data.columns:
        if col != 'geology_id' and not col.startswith('raw_'):
            feature_data[f'{col}_squared'] = feature_data[col] ** 2
            # Log transform for any potentially positive-only features
            if np.all(feature_data[col] > 0):
                feature_data[f'{col}_log'] = np.log1p(feature_data[col])
    
    # Add the original features as well
    for col in historical_cols:
        feature_data[col] = data[col]
    
    return feature_data

In [8]:
class LSTMForecastModel(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        output_size,
        dropout=0.2,
    ):
        super().__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
        )
        
        self.layer_norm = nn.LayerNorm(hidden_size)
        
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.activation = nn.GELU()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        
        lstm_out = lstm_out[:, -1, :]
        
        lstm_out = self.layer_norm(lstm_out)
        
        x = self.fc1(lstm_out)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [9]:
class TransformerForecastModel(nn.Module):
    def __init__(
        self,
        input_size,
        d_model=512,
        nhead=8,
        num_layers=4,
        dim_feedforward=2048,
        dropout=0.1,
        output_size=300
    ):
        super().__init__()
        
        self.input_projection = nn.Linear(input_size, d_model)
        
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        self.output_layer = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, output_size)
        )
        
    def forward(self, x):
        
        x = self.input_projection(x)
        
        x = self.pos_encoder(x)
        
        transformer_output = self.transformer_encoder(x)

        output = self.output_layer(transformer_output[:, -1, :])
        
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1)]
        return self.dropout(x)

In [10]:
class TCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout=0.2):
        super().__init__()
        
        # Padding для сохранения размерности
        padding = (kernel_size - 1) * dilation
        
        self.conv = nn.Conv1d(
            in_channels, 
            out_channels, 
            kernel_size, 
            padding=padding, 
            dilation=dilation
        )
        
        self.relu = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        
        self.residual = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()
        
        self.layer_norm = nn.LayerNorm(out_channels)
        
    def forward(self, x):
        residual = self.residual(x)
        
        out = self.conv(x)
        
        out = out[:, :, :-self.conv.padding[0]]
        
        out = out + residual
        
        out = self.relu(out)
        out = self.dropout(out)
        
        out = out.permute(0, 2, 1)
        out = self.layer_norm(out)
        out = out.permute(0, 2, 1) 
        
        return out

In [11]:
class TCNForecastModel(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size=128,
        kernel_size=3,
        num_layers=8,
        dropout=0.2,
        output_size=300
    ):
        super().__init__()
        
        self.input_projection = nn.Linear(input_size, hidden_size)
        
        self.tcn_blocks = nn.ModuleList()
        for i in range(num_layers):
            dilation = 2 ** i  # 1, 2, 4, 8, ...
            self.tcn_blocks.append(
                TCNBlock(hidden_size, hidden_size, kernel_size, dilation, dropout)
            )
        
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, output_size)
        )
        
    def forward(self, x):
        x = self.input_projection(x)
        
        x = x.permute(0, 2, 1)
        
        for block in self.tcn_blocks:
            x = block(x)
        
        x = x[:, :, -1]
        
        x = self.output_layer(x)
        
        return x

In [12]:
class HybridCNNLSTMModel(nn.Module):
    def __init__(
        self,
        input_size,
        cnn_filters=[64, 128, 128, 256],
        kernel_size=3,
        lstm_hidden=512,
        lstm_layers=2,
        dropout=0.2,
        output_size=300
    ):
        super().__init__()
        
        self.cnn_layers = nn.ModuleList()
        
        self.cnn_layers.append(nn.Conv1d(input_size, cnn_filters[0], kernel_size, padding=kernel_size//2))
        
        for i in range(1, len(cnn_filters)):
            self.cnn_layers.append(
                nn.Conv1d(cnn_filters[i-1], cnn_filters[i], kernel_size, padding=kernel_size//2)
            )
        
        self.batch_norms = nn.ModuleList([
            nn.BatchNorm1d(filters) for filters in cnn_filters
        ])
        
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        
        self.lstm = nn.LSTM(
            input_size=cnn_filters[-1],
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            dropout=dropout if lstm_layers > 1 else 0,
            bidirectional=True 
        )
        
        self.layer_norm = nn.LayerNorm(lstm_hidden * 2) 
        
        self.output_layer = nn.Sequential(
            nn.Linear(lstm_hidden * 2, lstm_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_hidden, output_size)
        )
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        for i, (conv, bn) in enumerate(zip(self.cnn_layers, self.batch_norms)):
            x = conv(x)
            x = bn(x)
            x = self.act(x)
            x = self.dropout(x)
        
        x = x.permute(0, 2, 1)
        
        lstm_out, _ = self.lstm(x)
        
        lstm_out = lstm_out[:, -1, :]
        
        lstm_out = self.layer_norm(lstm_out)

        x = self.output_layer(lstm_out)
        
        return x

In [13]:
class GeologyDataset(Dataset):
    def __init__(self, features, targets=None, is_test=False, scale_features=True):
        self.features = features
        self.targets = targets
        self.is_test = is_test
        
        if scale_features:
            self.feature_scaler = StandardScaler()
            self.features = self.feature_scaler.fit_transform(self.features)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        
        x = x.reshape(-1, 1)
        
        if self.is_test:
            return x
        else:
            y = self.targets[idx]
            return x, y


In [14]:
def preprocess_data(df, engineered_features=None, target_cols=None, is_test=False):
    if engineered_features is None:
        # If no engineered features provided, use raw data
        feature_cols = [c for c in df.columns if c != 'geology_id' and (c.startswith('-') or c == '0')]
        X = df[feature_cols].values
    else:
        # Use the engineered features, dropping the ID column
        X = engineered_features.drop('geology_id', axis=1).values
    
    if not is_test:
        y = df[target_cols].values
        return X, y
    
    return X

In [15]:
def compute_nll_score(solution, submission, row_id_column_name='geology_id'):
    solution_copy = solution.copy()
    submission_copy = submission.copy()
    
    del solution_copy[row_id_column_name]
    del submission_copy[row_id_column_name]

    NEGATIVE_PART = -299
    LARGEST_CHUNK = 600
    SMALLEST_CHUNK = 350
    TOTAL_REALIZATIONS = 10
    INFLATION_SIGMA = 600
    
    sigma_2 = np.ones((LARGEST_CHUNK+NEGATIVE_PART-1))
    from_ranges = [1, 61, 245]
    to_ranges_excl = [61, 245, 301]
    log_slopes = [1.0406028049510443, 0.0, 7.835345062351012]
    log_offsets = [-6.430669850650689, -2.1617411566043896, -45.24876794412965]

    for growth_mode in range(len(from_ranges)):
        for i in range(from_ranges[growth_mode], to_ranges_excl[growth_mode]):
            sigma_2[i-1] = np.exp(np.log(i)*log_slopes[growth_mode]+log_offsets[growth_mode])

    sigma_2 *= INFLATION_SIGMA
  
    cov_matrix_inv_diag = 1. / sigma_2
    
    num_rows = solution_copy.shape[0]
    num_columns = LARGEST_CHUNK + NEGATIVE_PART - 1
    
    p = 1./TOTAL_REALIZATIONS
    log_p = np.log(p)
    
    solution_arr = np.zeros((num_rows, TOTAL_REALIZATIONS, num_columns))
    submission_arr = np.zeros((num_rows, TOTAL_REALIZATIONS, num_columns))
    
    for k in range(TOTAL_REALIZATIONS):
        for i in range(num_columns):
            if k == 0:
                column_name = str(i+1)
            else:
                column_name = f"r_{k}_pos_{i+1}"
            solution_arr[:, k, i] = solution_copy[column_name].values
            submission_arr[:, k, i] = submission_copy[column_name].values

    misfit = solution_arr - submission_arr
    inner_product_matrix = np.sum(cov_matrix_inv_diag * misfit * misfit, axis=2)
    
    nll = -logsumexp(log_p - inner_product_matrix, axis=1)
    
    return nll.mean()

In [16]:
def train_model_with_nll_loss(model, train_loader, optimizer, device, epoch=0, total_epochs=30):
    model.train()
    train_losses = []
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{total_epochs}")
    
    for data, target in pbar:
        data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.float32)
        
        optimizer.zero_grad()
        output = model(data)
        
        target_mean = target.mean(dim=0)
        target_std = target.std(dim=0) + 1e-6
        
        normalized_output = (output - target_mean) / target_std
        normalized_target = (target - target_mean) / target_std
        
        loss = F.mse_loss(normalized_output, normalized_target)
        
        if output.shape[1] > 1:
            smoothness_penalty = torch.mean(torch.abs(output[:, 1:] - output[:, :-1]))
            loss += 0.01 * smoothness_penalty
        
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        train_losses.append(loss.item())
        pbar.set_postfix({'loss': f"{loss.item():.6f}"})
    
    return np.mean(train_losses)

In [17]:
def validate_model(model, val_loader, device):
    model.eval()
    val_losses = []
    val_preds = []
    val_targets = []
    
    with torch.no_grad():
        for data, target in tqdm(val_loader, desc="Validating"):
            data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.float32)
            output = model(data)
            
            target_mean = target.mean(dim=0)
            target_std = target.std(dim=0) + 1e-6
            
            normalized_output = (output - target_mean) / target_std
            normalized_target = (target - target_mean) / target_std
            
            loss = F.mse_loss(normalized_output, normalized_target)
            
            val_losses.append(loss.item())
            val_preds.append(output.cpu().numpy())
            val_targets.append(target.cpu().numpy())
    
    val_preds = np.concatenate(val_preds)
    val_targets = np.concatenate(val_targets)
    
    return np.mean(val_losses), val_preds, val_targets

In [18]:
def generate_diverse_realizations(base_predictions, num_samples=10, diversity_factor=0.2):
    num_rows, num_cols = base_predictions.shape
    realizations = np.zeros((num_samples, num_rows, num_cols))

    realizations[0] = base_predictions

    for i in range(1, num_samples):
        realization = base_predictions.copy()
        
        for j in range(num_rows):
            noise = np.random.normal(0, diversity_factor, num_cols)
            
            smoothed_noise = gaussian_filter1d(noise, sigma=5.0)
            
            position_factor = np.linspace(0.1, 1.0, num_cols)
            scaled_noise = smoothed_noise * position_factor
            
            realization[j] += scaled_noise

            for k in range(1, num_cols):

                max_change = 2.0 * (k/num_cols + 0.1)  # Allow larger changes further away
                diff = realization[j, k] - realization[j, k-1]
                if abs(diff) > max_change:
                    realization[j, k] = realization[j, k-1] + np.sign(diff) * max_change
        
        realizations[i] = realization
    
    return realizations

In [19]:
def generate_fold_realizations(base_predictions, num_realizations=10):
    realizations = generate_diverse_realizations(
        base_predictions, 
        num_samples=num_realizations,
        diversity_factor=0.15  # Control the diversity level
    )
    return realizations

In [20]:
def visualize_features(train_features, y):
    feature_cols = [col for col in train_features.columns if col != 'geology_id']
    
    target_cols = [str(i) for i in range(1, 11)]
    target_cols = [col for col in target_cols if col in train.columns]
    
    correlations = []
    for tcol in target_cols:
        if tcol in train.columns:
            for fcol in feature_cols:
                corr = np.corrcoef(train_features[fcol], train[tcol])[0, 1]
                correlations.append((fcol, tcol, abs(corr)))
    
    top_correlations = sorted(correlations, key=lambda x: x[2], reverse=True)[:15]
    
    plt.figure(figsize=(12, 8))
    plot_data = pd.DataFrame(top_correlations, columns=['Feature', 'Target', 'Correlation'])
    sns.barplot(data=plot_data, x='Correlation', y='Feature', hue='Target')
    plt.title('Top Feature Correlations with Targets')
    plt.tight_layout()
    
    try:
        wandb.log({"feature_correlations": wandb.Image(plt)})
    except:
        pass
    
    plt.close()

In [21]:
def get_default_config(model_type):    
    if model_type == 'LSTM':
        return {
            'model_type': 'LSTM',
            'hidden_size': 512,
            'num_layers': 2,
            'dropout': 0.2,
            'learning_rate': 5e-4,
            'weight_decay': 1e-5,
            'batch_size': 128,
            'epochs': 50,
            'seed': SEED,
            'feature_engineering': 'advanced',
            'optimizer': 'adamw',
            'scheduler': 'onecycle'
        }
    elif model_type == 'Transformer':
        return {
            'model_type': 'Transformer',
            'd_model': 512,
            'nhead': 8,
            'num_layers': 4,
            'dim_feedforward': 1024,
            'dropout': 0.2,
            'learning_rate': 4e-4,
            'weight_decay': 1e-5,
            'batch_size': 128,
            'epochs': 50,
            'seed': SEED,
            'feature_engineering': 'advanced',
            'optimizer': 'adamw',
            'scheduler': 'cosine'
        }
    elif model_type == 'TCN':
        return {
            'model_type': 'TCN',
            'hidden_size': 256,
            'kernel_size': 3,
            'num_layers': 8,
            'dropout': 0.2,
            'learning_rate': 5e-4,
            'weight_decay': 1e-5,
            'batch_size': 128,
            'epochs': 50,
            'seed': SEED,
            'feature_engineering': 'advanced',
            'optimizer': 'adam',
            'scheduler': 'onecycle'
        }
    elif model_type == 'HybridCNNLSTM':
        return {
            'model_type': 'HybridCNNLSTM',
            'cnn_filters': [64, 128, 128, 256],
            'kernel_size': 3,
            'hidden_size': 512,
            'num_layers': 2,
            'dropout': 0.3,
            'learning_rate': 3e-4,
            'weight_decay': 1e-5,
            'batch_size': 128,
            'epochs': 60,
            'seed': SEED,
            'feature_engineering': 'advanced',
            'optimizer': 'adamw',
            'scheduler': 'cosine'
        }
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")

In [22]:
def create_model(config, device):
    model_type = config['model_type']
    input_features_size = 1 
    output_size = 3000
    
    if model_type == 'LSTM':
        model = LSTMForecastModel(
            input_size=input_features_size,
            hidden_size=config['hidden_size'],
            num_layers=config['num_layers'],
            output_size=output_size,
            dropout=config['dropout']
        )
    elif model_type == 'Transformer':
        model = TransformerForecastModel(
            input_size=input_features_size,
            d_model=config.get('d_model', 512),
            nhead=config.get('nhead', 8),
            num_layers=config['num_layers'],
            dim_feedforward=config.get('dim_feedforward', 2048),
            dropout=config['dropout'],
            output_size=output_size
        )
    elif model_type == 'TCN':
        model = TCNForecastModel(
            input_size=input_features_size,
            hidden_size=config['hidden_size'],
            kernel_size=config.get('kernel_size', 3),
            num_layers=config['num_layers'],
            dropout=config['dropout'],
            output_size=output_size
        )
    elif model_type == 'HybridCNNLSTM':
        model = HybridCNNLSTMModel(
            input_size=input_features_size,
            cnn_filters=config.get('cnn_filters', [64, 128, 128, 256]),
            kernel_size=config.get('kernel_size', 3),
            lstm_hidden=config['hidden_size'],
            lstm_layers=config['num_layers'],
            dropout=config['dropout'],
            output_size=output_size
        )
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")
    
    return model.to(device)

In [23]:
def train_and_predict_single_model(
    X_train, 
    y_train,
    X_val,
    y_val,
    X_test,
    config,
    run_name=None,
    autofinish=True,
    wandb_run=None
):
    try:
        # Используем переданный wandb_run или инициализируем новый
        run = wandb_run if wandb_run is not None else init_wandb(project_name=run_name, config=config)
        
        train_dataset = GeologyDataset(X_train, y_train)
        val_dataset = GeologyDataset(X_val, y_val)
        test_dataset = GeologyDataset(X_test, is_test=True)
        
        train_loader = DataLoader(
            train_dataset, 
            batch_size=config['batch_size'], 
            shuffle=True,
            pin_memory=True, 
            num_workers=2  
        )
        val_loader = DataLoader(
            val_dataset, 
            batch_size=config['batch_size'], 
            shuffle=False,
            pin_memory=True,
            num_workers=2
        )
        test_loader = DataLoader(
            test_dataset, 
            batch_size=config['batch_size'], 
            shuffle=False,
            pin_memory=True,
            num_workers=2
        )
        
        model = create_model(config, device)
        
        if config.get('optimizer', 'adamw').lower() == 'adamw':
            optimizer = optim.AdamW(
                model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config['weight_decay'],
                eps=1e-8
            )
        elif config.get('optimizer', 'adamw').lower() == 'adam':
            optimizer = optim.Adam(
                model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config['weight_decay'],
                eps=1e-8
            )
        elif config.get('optimizer', 'adamw').lower() == 'sgd':
            optimizer = optim.SGD(
                model.parameters(),
                lr=config['learning_rate'],
                momentum=config.get('momentum', 0.9),
                weight_decay=config['weight_decay']
            )
        
        if config.get('scheduler', 'onecycle').lower() == 'onecycle':
            steps_per_epoch = len(train_loader)
            scheduler = optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=config['learning_rate'],
                epochs=config['epochs'],
                steps_per_epoch=steps_per_epoch,
                pct_start=0.3,
                div_factor=25,
                final_div_factor=1000,
            )
        elif config.get('scheduler', 'onecycle').lower() == 'cosine':
            scheduler = optim.lr_scheduler.CosineAnnealingLR(
                optimizer, 
                T_max=config['epochs'] // 2,
                eta_min=config['learning_rate'] / 1000
            )
        elif config.get('scheduler', 'onecycle').lower() == 'reduce':
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                mode='min',
                factor=0.5,
                patience=5,
                min_lr=config['learning_rate'] / 100
            )
        
        best_val_loss = float('inf')
        val_predictions = None
        
        # early stopping
        patience = 10
        counter = 0
        min_delta = 1e-4
        
        print(f"Training model {config['model_type']}...")
        for epoch in range(config['epochs']):
            train_loss = train_model_with_nll_loss(
                model, train_loader, optimizer, device, epoch, config['epochs']
            )
            
            val_loss, val_preds, val_targets = validate_model(model, val_loader, device)
            
            val_predictions = val_preds
            
            if config.get('scheduler', 'onecycle').lower() in ['onecycle', 'cosine']:
                scheduler.step()
            elif config.get('scheduler', 'onecycle').lower() == 'reduce':
                scheduler.step(val_loss)
    
            if run:
                run.log({
                    "epoch": epoch + 1,
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "learning_rate": optimizer.param_groups[0]['lr']
                })
            
            print(f"Epoch {epoch+1}/{config['epochs']} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
            
            # loss check
            if epoch + 1 == 12 and val_loss > 1.0:
                print(f"Validation loss > 1.0 at epoch 10 ({val_loss:.6f}). Stopping training.")
                if run:
                    run.log({"early_stop_reason": "high_loss_at_epoch_10", "best_val_loss": best_val_loss})
                break
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                model_path = f"model_{config['model_type']}.pt"
                torch.save(model.state_dict(), model_path)
                if run:
                    run.save(model_path)
                print(f"Saved new best model with validation loss: {val_loss:.6f}")
                counter = 0 
            else:
                if val_loss > best_val_loss - min_delta:
                    counter += 1
                    if counter >= patience:
                        print(f"Early stopping triggered at epoch {epoch+1}. Best val_loss: {best_val_loss:.6f}")
                        if run:
                            run.log({"early_stop_reason": "no_improvement", "best_val_loss": best_val_loss})
                        break
        
        model.load_state_dict(torch.load(f"model_{config['model_type']}.pt"))
        model.eval()
        test_preds = []
        
        with torch.no_grad():
            for data in tqdm(test_loader, desc="Predicting test data"):
                if isinstance(data, list):
                    data = data[0]
                data = data.to(device, dtype=torch.float32)
                output = model(data)
                test_preds.append(output.cpu().numpy())
        
        base_test_predictions = np.concatenate(test_preds)
        
        val_preds_df = pd.DataFrame(
            data=val_predictions,
            columns=TARGETS,
        )
        val_preds_df['geology_id'] = val_dataset.features[:, 0]
        
        val_solution_df = pd.DataFrame(
            data=y_val,
            columns=TARGETS,
        )
        val_solution_df['geology_id'] = val_dataset.features[:, 0]
        
        nll_score = compute_nll_score(val_solution_df, val_preds_df)
        
        # Завершаем run только если мы его создали и autofinish=True
        if run and autofinish and wandb_run is None:
            run.log({"val_nll_score": nll_score})
            run.finish()
        
        return base_test_predictions, val_predictions, nll_score
        
    finally:
        torch.cuda.empty_cache()
        
        from torch.utils.data import _utils
        if hasattr(_utils.worker, "_worker_pool"):
            _utils.worker._worker_pool.close()
            _utils.worker._worker_pool.join()
        elif hasattr(_utils.worker, "_shutdown_all_workers"):
            _utils.worker._shutdown_all_workers()
        
        import gc
        gc.collect()

In [24]:
def create_submission(base_predictions, config, val_nll_score):
    submission = sub.copy()
    
    for i in range(300):
        col_name = str(i+1)
        submission[col_name] = base_predictions[:, i]
    
    realizations = generate_diverse_realizations(
        base_predictions, 
        num_samples=10, 
        diversity_factor=0.15 + 0.05 * (config.get('diversity_factor', 1.0) - 1.0)
    )
    
    for r_idx in range(1, 10): 
        for i in range(300):
            col_name = f"r_{r_idx}_pos_{i+1}"
            submission[col_name] = realizations[r_idx][:, i]
    
    submission_file = f"submission_{config['model_type']}_{val_nll_score:.6f}.csv"
    submission.to_csv(submission_file, index=False)
    print(f"\nSubmission file saved: {submission_file}")
    
    expected_cols = sub.columns.tolist()
    actual_cols = submission.columns.tolist()
    
    if set(expected_cols) != set(actual_cols):
        print("WARNING: Submission columns don't match expected format!")
        missing = set(expected_cols) - set(actual_cols)
        extra = set(actual_cols) - set(expected_cols)
        if missing:
            print(f"Missing columns: {missing}")
        if extra:
            print(f"Extra columns: {extra}")
    else:
        print("Submission format validated successfully!")
    
    return submission

In [25]:
def visualize_model_predictions(submission, config, run=None):
    try:
        plt.figure(figsize=(15, 8))

        sample_indices = [0, 10, 20]
        
        for sample_idx in sample_indices:
            plt.figure(figsize=(15, 8))
            
            x_coords = np.arange(1, 301)
            plt.plot(x_coords, submission.iloc[sample_idx, 1:301].values, 
                    color='blue', label='Baseline', linewidth=2)
            
            colors = ['red', 'green', 'purple']
            for r in range(1, 4): 
                cols = [f"r_{r}_pos_{i+1}" for i in range(300)]
                plt.plot(x_coords, submission.loc[submission.index[sample_idx], cols].values,
                        color=colors[(r-1) % len(colors)], label=f'Realization {r}', alpha=0.7)
            
            plt.title(f"{config['model_type']} - Sample {sample_idx}")
            plt.xlabel("Position")
            plt.ylabel("Layer Depth (Z coordinate)")
            plt.legend()
            plt.grid(True, alpha=0.3)

            if run:
                run.log({f"predictions_sample_{sample_idx}": wandb.Image(plt)})
            
            plt.close()
        
    except Exception as e:
        print(f"Visualization error: {e}")

In [26]:
def split_train_val(X_features, y, val_size=0.2, random_state=42):
    n_samples = len(X_features)
    indices = np.arange(n_samples)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    val_samples = int(val_size * n_samples)
    val_indices = indices[:val_samples]
    train_indices = indices[val_samples:]
    
    X_train = X_features[train_indices]
    y_train = y[train_indices]
    X_val = X_features[val_indices]
    y_val = y[val_indices]
    
    return X_train, y_train, X_val, y_val, train_indices, val_indices

In [27]:
def run_experiment(config, autofinish=True, wandb_run=None):
    try:
        seed_everything(config['seed'])
        
        print(f"\n{'='*50}\nRunning experiment with {config['model_type']}\n{'='*50}")
        print(f"Config: {config}")
        
        X_train, y_train, X_val, y_val, train_indices, val_indices = split_train_val(
            X_features, y, val_size=0.2, random_state=config['seed']
        )

        base_predictions, val_predictions, val_nll_score = train_and_predict_single_model(
        X_train, y_train, X_val, y_val, X_features_test, config, 
        run_name="geology-forecast-challenge", 
        autofinish=autofinish, wandb_run=wandb_run)
        
        train_sub.loc[val_indices, TARGETS] = val_predictions
        
        submission = create_submission(base_predictions, config, val_nll_score)
    
        visualize_model_predictions(submission, config)
        
        print(f"Finished experiment with {config['model_type']}, validation NLL: {val_nll_score:.6f}")
        
        return val_nll_score
    
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(f"CUDA OOM error with config: {config}")
            
            if wandb.run is not None:
                wandb.log({"cuda_oom_error": True, "error_message": str(e)})
                
            # Большое значение метрики, чтобы байесовский оптимизатор избегал таких конфигураций
            return float('inf')
        else:
            raise e

In [28]:
def visualize_predictions(test_idx=0, num_realizations=3):
    plt.figure(figsize=(15, 8))
    
    x_coords = np.arange(1, 301)
    plt.plot(x_coords, submission.iloc[test_idx, 1:301].values, 
             color='blue', label='Realization 0', linewidth=2)
    
    colors = ['red', 'green', 'purple']
    for r in range(1, min(num_realizations+1, 10)):
        cols = [f"r_{r}_pos_{i+1}" for i in range(300)]
        plt.plot(x_coords, submission.loc[submission.index[test_idx], cols].values,
                color=colors[(r-1) % len(colors)], label=f'Realization {r}', alpha=0.7)
    
    plt.title(f"Multiple Geological Sequence Realizations for Sample {test_idx}")
    plt.xlabel("Position")
    plt.ylabel("Layer Depth (Z coordinate)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    try:
        wandb.log({"prediction_visualization": wandb.Image(plt)})
    except:
        pass
    
    plt.close()

In [29]:
def analyze_geological_patterns(submission_df, sample_indices=None, num_samples=5):
    if sample_indices is None:
        sample_indices = np.random.choice(len(submission_df), min(num_samples, len(submission_df)), replace=False)
    
    plt.figure(figsize=(18, 12))
    
    for idx, i in enumerate(sample_indices):
        plt.subplot(len(sample_indices), 1, idx+1)
        
        base_pred = submission_df.iloc[i, 1:301].values
        
        slopes = np.diff(base_pred)
        
        threshold = np.std(slopes) * 2.5
        fault_indicators = np.where(np.abs(slopes) > threshold)[0]
        
        plt.plot(range(1, 301), base_pred, 'b-', linewidth=2, label='Predicted Sequence')
        
        if len(fault_indicators) > 0:
            plt.scatter([x+1 for x in fault_indicators], 
                       [base_pred[x] for x in fault_indicators],
                       color='red', s=80, marker='x', label='Potential Fault/Change')
        
        segment_size = 50
        for seg_start in range(0, 300, segment_size):
            seg_end = min(seg_start + segment_size, 300)
            if seg_end - seg_start > 10:  # Only fit if enough points
                x_seg = np.arange(seg_start, seg_end)
                y_seg = base_pred[seg_start:seg_end]
                # Fit a line to this segment
                z = np.polyfit(x_seg, y_seg, 1)
                p = np.poly1d(z)
                plt.plot(x_seg+1, p(x_seg), '--', linewidth=1.5, 
                         alpha=0.7, label=f'Trend (Seg {seg_start}-{seg_end})')
        
        plt.title(f"Geological Analysis for Sample {i}")
        plt.xlabel("Position")
        plt.ylabel("Layer Depth (Z)")
        plt.grid(True, alpha=0.3)
        if idx == 0:
            plt.legend(loc='upper right')
    
    plt.tight_layout()

    try:
        wandb.log({"geological_analysis": wandb.Image(plt)})
    except:
        pass
    
    plt.close()

In [30]:
def calculate_prediction_uncertainty(submission_df, num_samples=5):
    sample_indices = np.random.choice(len(submission_df), min(num_samples, len(submission_df)), replace=False)
    
    plt.figure(figsize=(15, 10))
    
    for idx, i in enumerate(sample_indices):
        plt.subplot(num_samples, 1, idx+1)
        
        realizations = []
        
        base_realization = submission_df.iloc[i, 1:301].values
        realizations.append(base_realization)
        
        for r in range(1, 10):
            cols = [f"r_{r}_pos_{i+1}" for i in range(300)]
            realization = submission_df.loc[submission_df.index[i], cols].values
            realizations.append(realization)
        
        realizations = np.array(realizations)
        
        mean_prediction = np.mean(realizations, axis=0)
        std_prediction = np.std(realizations, axis=0)
        
        x_coords = np.arange(1, 301)
        plt.plot(x_coords, mean_prediction, 'b-', label='Mean Prediction')
        
        plt.fill_between(x_coords, 
                         mean_prediction - 2*std_prediction,
                         mean_prediction + 2*std_prediction,
                         alpha=0.3, color='blue',
                         label='95% Confidence Interval')
        
        high_uncertainty = np.where(std_prediction > np.mean(std_prediction) + np.std(std_prediction))[0]
        if len(high_uncertainty) > 0:
            plt.scatter(high_uncertainty+1, 
                       mean_prediction[high_uncertainty],
                       color='red', s=50, alpha=0.7, 
                       label='High Uncertainty Regions')
        
        plt.title(f"Prediction Uncertainty Analysis for Sample {i}")
        plt.xlabel("Position")
        plt.ylabel("Layer Depth (Z)")
        plt.grid(True, alpha=0.3)
        if idx == 0:
            plt.legend(loc='upper right')
    
    plt.tight_layout()
    
    try:
        wandb.log({"uncertainty_analysis": wandb.Image(plt)})
    except:
        pass
    
    plt.close()

In [31]:
def create_sweep_config():
    sweep_config = {
        'method': 'bayes', 
        'metric': {
            'name': 'val_nll_score',
            'goal': 'minimize'
        },
        'parameters': {
            'model_type': {
                'values': ['HybridCNNLSTM']
            },
            'learning_rate': {
                'distribution': 'log_uniform_values', 
                'min': 3e-4,
                'max': 1e-2
            },
            'batch_size': {
                'values': [128]
            },
            'dropout': {
                'distribution': 'uniform',
                'min': 0.1,
                'max': 0.3 
            },
            'hidden_size': {
                'values': [128]
            },
            'num_layers': {
                'values': [2, 3, 4]
            },
            'weight_decay': {
                'values': [1e-5, 1e-4]
            },
            'optimizer': {
                'values': ['adamw', 'adamw', 'sgd']
            },
            'scheduler': {
                'values': ['cosine', 'reduce', 'onecycle']
            },
            'diversity_factor': {
                'distribution': 'uniform',
                'min': 0.8,
                'max': 1.5
            }
        }
    }
    
    sweep_config['parameters']['d_model'] = {
        'values': [128]
    }
    sweep_config['parameters']['nhead'] = {
        'values': [4, 8, 12]
    }
    sweep_config['parameters']['dim_feedforward'] = {
        'values': [128, 256]
    }
    
    sweep_config['parameters']['kernel_size'] = {
        'values': [3, 5]
    }
    
    return sweep_config

In [32]:
def sweep_agent():
    global X_features, y, X_features_test
    
    # Инициализируем wandb run и сохраняем объект run
    run = wandb.init(reinit=True)
    
    config = wandb.config
    model_type = config['model_type']
    
    default_config = get_default_config(model_type)
    
    experiment_config = dict(config)
    
    for key, value in default_config.items():
        if key not in experiment_config:
            experiment_config[key] = value
    
    transformer_params = ['d_model', 'nhead', 'dim_feedforward']
    tcn_params = ['kernel_size']
    
    if model_type != 'Transformer':
        for param in transformer_params:
            if param in experiment_config: 
                del experiment_config[param]
    
    if model_type != 'TCN':
        for param in tcn_params:
            if param in experiment_config:
                del experiment_config[param]
    
    print(f"Running experiment with config: {experiment_config}")
    
    # Передаем объект run в run_experiment
    val_nll_score = run_experiment(experiment_config, autofinish=False, wandb_run=run)
    
    # Логируем с использованием того же объекта run
    run.log({'val_nll_score': val_nll_score, 'final_val_nll_score': val_nll_score})
    
    # wandb.finish() не нужен, так как агент сам позаботится о завершении
    
    return val_nll_score

In [33]:
def run_sweep(count=10):
    # Инициализируем начальный run для создания sweep
    init_wandb(project_name="geology-forecast-challenge-sweep-gpu-bayes-30-server-HybridCNNLSTM")
    
    sweep_config = create_sweep_config()
    sweep_id = wandb.sweep(sweep_config, project="geology-forecast-challenge-sweep-gpu-bayes-30-server-HybridCNNLSTM")
    
    # Завершаем начальный run перед запуском агента
    wandb.finish()
    
    # Запускаем агента
    wandb.agent(sweep_id, function=sweep_agent, count=count)

In [34]:
def run_single_model(model_type='HybridCNNLSTM', custom_params=None):
    config = get_default_config(model_type)
    
    if custom_params:
        config.update(custom_params)
    
    val_nll_score = run_experiment(config)
    
    return val_nll_score, config

In [35]:
"""
score, config = run_single_model('HybridCNNLSTM', {
    'hidden_size': 512, 
    'num_layers': 2,
    'dropout': 0.3,
    'learning_rate': 3e-4,
    'epochs': 60 
})

print(f"Final validation NLL score: {score:.6f}")
"""

'\nscore, config = run_single_model(\'HybridCNNLSTM\', {\n    \'hidden_size\': 512, \n    \'num_layers\': 2,\n    \'dropout\': 0.3,\n    \'learning_rate\': 3e-4,\n    \'epochs\': 60 \n})\n\nprint(f"Final validation NLL score: {score:.6f}")\n'

In [36]:
print("Engineering features for train data...")
train_features = engineer_features(train)

print("Engineering features for test data...")
test_features = engineer_features(test)

X_features = train_features.drop('geology_id', axis=1).values
y = train[TARGETS].values
X_features_test = test_features.drop('geology_id', axis=1).values

print(f"Feature shape: {X_features.shape}, Target shape: {y.shape}")

Engineering features for train data...
Engineering features for test data...
Feature shape: (1510, 397), Target shape: (1510, 3000)


In [38]:
!pip install xgboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 580.6/580.6 kB 30.4 MB/s eta 0:00:00


In [43]:
import xgboost as xgb
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
from optuna.samplers import TPESampler

def train_xgboost_model(X_train, y_train, X_val=None, y_val=None, params=None, num_boost_round=1000, early_stopping_rounds=50):
    """
    Train XGBoost model with early stopping
    """
    if params is None:
        params = get_default_xgb_params()
    
    # Create DMatrix objects for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    # Create validation set if provided
    eval_list = [(dtrain, 'train')]
    if X_val is not None and y_val is not None:
        dval = xgb.DMatrix(X_val, label=y_val)
        eval_list.append((dval, 'valid'))
    
    # Train model with early stopping
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=eval_list,
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=100
    )
    
    return model

def get_default_xgb_params():
    """
    Default XGBoost parameters
    """
    return {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'learning_rate': 0.01,
        'max_depth': 6,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.01,
        'reg_lambda': 1,
        'gamma': 0,
        'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'hist',
        'predictor': 'gpu_predictor' if torch.cuda.is_available() else 'cpu_predictor',
        'seed': SEED
    }

def predict_output_sequence(model, X):
    """
    Generate predictions for each target position using a single XGBoost model
    """
    dtest = xgb.DMatrix(X)
    return model.predict(dtest)

def cross_validate_xgb(X, y, params, n_splits=5, target_cols=None):
    """
    Cross-validate XGBoost model
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    cv_scores = []
    oof_preds = np.zeros_like(y)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\n--- Fold {fold+1}/{n_splits} ---")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Train the model
        model = train_xgboost_model(X_train, y_train, X_val, y_val, params)
        
        # Make predictions
        val_preds = predict_output_sequence(model, X_val)
        oof_preds[val_idx] = val_preds
        
        # Calculate fold score
        mse = mean_squared_error(y_val, val_preds)
        print(f"Fold {fold+1} MSE: {mse:.6f}")
        cv_scores.append(mse)
    
    # Calculate overall CV score
    cv_score = np.mean(cv_scores)
    print(f"\nCV MSE: {cv_score:.6f}")
    
    return cv_score, oof_preds

def xgb_multioutput_training(X, y, X_test, params=None, n_splits=5):
    """
    Train a set of XGBoost models, one for each geological position
    """
    if params is None:
        params = get_default_xgb_params()
    
    # Select positions to model (we'll train models for each position)
    # For computational efficiency, we'll train models for positions at intervals
    # and interpolate the rest
    interval = 5  # Model every 5th position
    positions_to_model = list(range(0, 300, interval))
    if 299 not in positions_to_model:  # Make sure we model the last position
        positions_to_model.append(299)
    
    models = {}
    oof_preds = np.zeros((X.shape[0], 300))
    test_preds = np.zeros((X_test.shape[0], 300))
    
    # Initialize W&B run
    run = init_wandb(project_name="geology-forecast-challenge-xgb", config=params)
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    for pos in tqdm(positions_to_model, desc="Training position models"):
        pos_models = []
        pos_oof_preds = np.zeros(X.shape[0])
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            X_train, X_val = X[train_idx], X[val_idx]
            # Extract target for current position
            y_train = y[train_idx, pos]
            y_val = y[val_idx, pos]
            
            # Train model
            pos_model = train_xgboost_model(
                X_train, y_train, X_val, y_val, params, 
                num_boost_round=2000, early_stopping_rounds=50
            )
            
            # Save model
            pos_models.append(pos_model)
            
            # Generate OOF predictions
            val_preds = predict_output_sequence(pos_model, X_val)
            pos_oof_preds[val_idx] = val_preds
            
        # Store model for this position
        models[pos] = pos_models
        
        # Store OOF predictions for this position
        oof_preds[:, pos] = pos_oof_preds
        
        # Generate test predictions (average of fold models)
        pos_test_preds = np.zeros(X_test.shape[0])
        for model in pos_models:
            pos_test_preds += predict_output_sequence(model, X_test) / len(pos_models)
        test_preds[:, pos] = pos_test_preds
        
        if run:
            mse = mean_squared_error(y[:, pos], pos_oof_preds)
            run.log({f"position_{pos}_mse": mse})
    
    # Interpolate predictions for positions not explicitly modeled
    for i in range(300):
        if i in positions_to_model:
            continue
        
        # Find the closest modeled positions
        left_pos = max([p for p in positions_to_model if p < i])
        right_pos = min([p for p in positions_to_model if p > i])
        
        # Calculate interpolation weights
        total_dist = right_pos - left_pos
        left_weight = (right_pos - i) / total_dist
        right_weight = (i - left_pos) / total_dist
        
        # Interpolate OOF predictions
        oof_preds[:, i] = left_weight * oof_preds[:, left_pos] + right_weight * oof_preds[:, right_pos]
        
        # Interpolate test predictions
        test_preds[:, i] = left_weight * test_preds[:, left_pos] + right_weight * test_preds[:, right_pos]
    
    if run:
        mse = mean_squared_error(y, oof_preds)
        run.log({"overall_mse": mse})
    
    return models, oof_preds, test_preds

def nll_cv_score(oof_preds, y, geology_ids):
    """
    Calculate NLL score for cross-validation
    """
    # Create DataFrames with the required format for NLL score
    oof_df = pd.DataFrame({"geology_id": geology_ids})
    solution_df = pd.DataFrame({"geology_id": geology_ids})
    
    # Add base predictions
    for i in range(300):
        col_name = str(i+1)
        oof_df[col_name] = oof_preds[:, i]
        solution_df[col_name] = y[:, i]
    
    # Generate diverse realizations
    realizations = generate_diverse_realizations(oof_preds, num_samples=10, diversity_factor=0.15)
    
    # Add realizations to oof_df
    for r_idx in range(1, 10):
        for i in range(300):
            col_name = f"r_{r_idx}_pos_{i+1}"
            oof_df[col_name] = realizations[r_idx][:, i]
            solution_df[col_name] = y[:, i]
    
    # Calculate NLL score
    nll = compute_nll_score(solution_df, oof_df)
    return nll

def generate_geological_features(X):
    """
    Generate additional geological-domain specific features
    """
    # Extract the last 50 points for each sample
    historical_features = []
    
    # Find columns that start with 'raw_'
    raw_cols = [col for col in range(X.shape[1]) if col >= X.shape[1] - 50]
    
    if len(raw_cols) >= 10:
        # Get the raw values
        raw_data = X[:, raw_cols]
        
        # Calculate local slopes (first derivative)
        slopes = np.diff(raw_data, axis=1)
        mean_slope = np.mean(slopes, axis=1).reshape(-1, 1)
        std_slope = np.std(slopes, axis=1).reshape(-1, 1)
        max_slope = np.max(np.abs(slopes), axis=1).reshape(-1, 1)
        
        # Calculate curvature (second derivative)
        curvature = np.diff(slopes, axis=1)
        mean_curv = np.mean(curvature, axis=1).reshape(-1, 1)
        std_curv = np.std(curvature, axis=1).reshape(-1, 1)
        max_curv = np.max(np.abs(curvature), axis=1).reshape(-1, 1)
        
        # Create features from wavelets
        from scipy import signal
        wavelet_features = []
        for wl in ['haar', 'db1', 'sym2']:
            try:
                coeffs = np.array([signal.cwt(row, signal.ricker, [2, 4, 8, 16]) for row in raw_data])
                wavelet_stats = np.hstack([
                    np.mean(coeffs, axis=(1, 2)).reshape(-1, 1),
                    np.std(coeffs, axis=(1, 2)).reshape(-1, 1),
                    np.max(coeffs, axis=(1, 2)).reshape(-1, 1),
                    np.min(coeffs, axis=(1, 2)).reshape(-1, 1)
                ])
                wavelet_features.append(wavelet_stats)
            except:
                pass
        
        if wavelet_features:
            wavelet_features = np.hstack(wavelet_features)
            
            # Combine all features
            geo_features = np.hstack([
                mean_slope, std_slope, max_slope,
                mean_curv, std_curv, max_curv,
                wavelet_features
            ])
            
            # Concatenate original features with new ones
            X_enhanced = np.hstack([X, geo_features])
            return X_enhanced
    
    return X

class OptimizeXGBoost:
    """
    Optuna optimization for XGBoost hyperparameters
    """
    def __init__(self, X, y, n_trials=50):
        self.X = X
        self.y = y
        self.n_trials = n_trials
        self.best_params = None
        self.best_score = float('inf')
        
    def objective(self, trial):
        # Define hyperparameters to optimize
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
            'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
            'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'hist',
            'predictor': 'gpu_predictor' if torch.cuda.is_available() else 'cpu_predictor',
            'seed': SEED
        }
        
        # Perform k-fold cross validation with current parameters
        kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
        
        # We'll sample a subset of target positions for efficiency
        sampled_positions = np.random.choice(range(300), size=10, replace=False)
        
        scores = []
        for pos in sampled_positions:
            pos_scores = []
            
            for train_idx, val_idx in kf.split(self.X):
                X_train, X_val = self.X[train_idx], self.X[val_idx]
                y_train, y_val = self.y[train_idx, pos], self.y[val_idx, pos]
                
                model = train_xgboost_model(X_train, y_train, X_val, y_val, params, 
                                           num_boost_round=1000, early_stopping_rounds=50)
                
                val_preds = predict_output_sequence(model, X_val)
                
                mse = mean_squared_error(y_val, val_preds)
                pos_scores.append(mse)
            
            scores.append(np.mean(pos_scores))
        
        mean_score = np.mean(scores)
        
        if mean_score < self.best_score:
            self.best_score = mean_score
            self.best_params = params
        
        return mean_score
    
    def optimize(self):
        wandb_kwargs = {"project": "geology-forecast-xgb-optuna"}
        wandb_callback = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
        
        sampler = TPESampler(seed=SEED)
        study = optuna.create_study(direction='minimize', sampler=sampler)
        study.optimize(self.objective, n_trials=self.n_trials, callbacks=[wandb_callback])
        
        print(f"Best trial: {study.best_trial.number}")
        print(f"Best MSE: {study.best_value:.6f}")
        print(f"Best hyperparameters: {study.best_params}")
        
        return study.best_params

def xgb_pipeline(X_features, y, X_features_test, optimize=True, n_jobs=-1):
    """
    Complete XGBoost training pipeline with feature engineering and model tuning
    """
    print("Starting XGBoost pipeline...")
    
    # Original feature shape
    print(f"Original feature shape: {X_features.shape}")
    
    # Generate additional geological features
    X_features = generate_geological_features(X_features)
    X_features_test = generate_geological_features(X_features_test)
    
    print(f"Enhanced feature shape: {X_features.shape}")
    
    # Standardize features
    scaler = StandardScaler()
    X_features_scaled = scaler.fit_transform(X_features)
    X_features_test_scaled = scaler.transform(X_features_test)
    
    # Optimize hyperparameters
    if optimize:
        print("Optimizing XGBoost hyperparameters...")
        optimizer = OptimizeXGBoost(X_features_scaled, y, n_trials=1)
        best_params = optimizer.optimize()
    else:
        best_params = get_default_xgb_params()
    
    print(f"Training with parameters: {best_params}")
    
    # Train models for multiple positions
    models, oof_preds, test_preds = xgb_multioutput_training(
        X_features_scaled, y, X_features_test_scaled, 
        params=best_params, n_splits=5
    )
    
    # Calculate NLL score on OOF predictions
    geology_ids = train['geology_id'].values
    nll = nll_cv_score(oof_preds, y, geology_ids)
    print(f"OOF NLL Score: {nll:.6f}")
    
    # Create and save submission
    submission = create_submission_xgb(test_preds, best_params, nll)
    
    # Visualize results
    visualize_model_predictions(submission, {"model_type": "XGBoost"})
    
    return models, oof_preds, test_preds, nll, submission

def create_submission_xgb(base_predictions, config, val_nll_score):
    """
    Create submission file from XGBoost predictions
    """
    submission = sub.copy()
    
    for i in range(300):
        col_name = str(i+1)
        submission[col_name] = base_predictions[:, i]
    
    # Generate diverse realizations for different scenarios
    realizations = generate_diverse_realizations(
        base_predictions, 
        num_samples=10, 
        diversity_factor=0.15
    )
    
    for r_idx in range(1, 10): 
        for i in range(300):
            col_name = f"r_{r_idx}_pos_{i+1}"
            submission[col_name] = realizations[r_idx][:, i]
    
    submission_file = f"submission_XGBoost_{val_nll_score:.6f}.csv"
    submission.to_csv(submission_file, index=False)
    print(f"\nSubmission file saved: {submission_file}")
    
    expected_cols = sub.columns.tolist()
    actual_cols = submission.columns.tolist()
    
    if set(expected_cols) != set(actual_cols):
        print("WARNING: Submission columns don't match expected format!")
        missing = set(expected_cols) - set(actual_cols)
        extra = set(actual_cols) - set(expected_cols)
        if missing:
            print(f"Missing columns: {missing}")
        if extra:
            print(f"Extra columns: {extra}")
    else:
        print("Submission format validated successfully!")
    
    return submission

def visualize_feature_importance(models, feature_names):
    """
    Visualize feature importance across models
    """
    importance_df = pd.DataFrame()
    
    # Sample a few positions to analyze
    positions = list(models.keys())
    sample_positions = np.random.choice(positions, min(5, len(positions)), replace=False)
    
    for pos in sample_positions:
        # Get the first fold model for this position
        model = models[pos][0]
        
        # Get feature importance
        importance = model.get_score(importance_type='gain')
        
        # Convert to DataFrame
        pos_df = pd.DataFrame({
            'Feature': list(importance.keys()),
            'Importance': list(importance.values()),
            'Position': f'Pos {pos}'
        })
        
        importance_df = pd.concat([importance_df, pos_df])
    
    # Visualize top features
    plt.figure(figsize=(12, 8))
    
    # Get top 20 features by average importance
    top_features = importance_df.groupby('Feature')['Importance'].mean().nlargest(20).index
    
    # Filter to only these features
    plot_df = importance_df[importance_df['Feature'].isin(top_features)]
    
    sns.barplot(data=plot_df, x='Importance', y='Feature', hue='Position')
    plt.title('Feature Importance by Position')
    plt.tight_layout()
    
    try:
        wandb.log({"feature_importance": wandb.Image(plt)})
    except:
        pass
    
    plt.close()

In [40]:
pip install optuna-integration[wandb]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Run the full XGBoost pipeline
models, oof_preds, test_preds, nll_score, submission = xgb_pipeline(
    X_features, y, X_features_test, optimize=True
)

print(f"Final NLL Score: {nll_score:.6f}")

Starting XGBoost pipeline...
Original feature shape: (1510, 397)


[I 2025-04-26 15:21:37,744] A new study created in memory with name: no-name-af1f0f8d-88fc-4087-b05a-da0077c315e6


Enhanced feature shape: (1510, 415)
Optimizing XGBoost hyperparameters...
[0]	train-rmse:1.30884	valid-rmse:1.26892
[100]	train-rmse:0.80687	valid-rmse:0.78102
[200]	train-rmse:0.54287	valid-rmse:0.53492
[300]	train-rmse:0.41149	valid-rmse:0.42541
[400]	train-rmse:0.34765	valid-rmse:0.38591
[500]	train-rmse:0.31371	valid-rmse:0.37271
[600]	train-rmse:0.29039	valid-rmse:0.36742
[700]	train-rmse:0.27174	valid-rmse:0.36434
[800]	train-rmse:0.25709	valid-rmse:0.36347
[900]	train-rmse:0.24356	valid-rmse:0.36220
[999]	train-rmse:0.23168	valid-rmse:0.36175
[0]	train-rmse:1.32453	valid-rmse:1.20348
[100]	train-rmse:0.81748	valid-rmse:0.72238
[200]	train-rmse:0.55046	valid-rmse:0.48219
[300]	train-rmse:0.41795	valid-rmse:0.37905
[400]	train-rmse:0.35454	valid-rmse:0.34500
[500]	train-rmse:0.32090	valid-rmse:0.33687
[544]	train-rmse:0.30910	valid-rmse:0.33805
[0]	train-rmse:1.30813	valid-rmse:1.27200
[100]	train-rmse:0.80246	valid-rmse:0.80231
[200]	train-rmse:0.53629	valid-rmse:0.57054
[300]	tr

[I 2025-04-26 15:25:36,808] Trial 0 finished with value: 8.130498995293056 and parameters: {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'min_child_weight': 8, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.7699302940633311e-07, 'reg_lambda': 2.9152036385288193e-08, 'gamma': 0.08499808989182997}. Best is trial 0 with value: 8.130498995293056.


Best trial: 0
Best MSE: 8.130499
Best hyperparameters: {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'min_child_weight': 8, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.7699302940633311e-07, 'reg_lambda': 2.9152036385288193e-08, 'gamma': 0.08499808989182997}
Training with parameters: {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'min_child_weight': 8, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.7699302940633311e-07, 'reg_lambda': 2.9152036385288193e-08, 'gamma': 0.08499808989182997}


Training position models:   0%|          | 0/61 [00:00<?, ?it/s]

[0]	train-rmse:0.03573	valid-rmse:0.03661
[100]	train-rmse:0.02309	valid-rmse:0.02459
[200]	train-rmse:0.01734	valid-rmse:0.01924
[300]	train-rmse:0.01459	valid-rmse:0.01673
[400]	train-rmse:0.01438	valid-rmse:0.01654
[500]	train-rmse:0.01438	valid-rmse:0.01654
[534]	train-rmse:0.01438	valid-rmse:0.01654
[0]	train-rmse:0.03655	valid-rmse:0.03324
[100]	train-rmse:0.02374	valid-rmse:0.02019
[200]	train-rmse:0.01793	valid-rmse:0.01403
[300]	train-rmse:0.01508	valid-rmse:0.01098
[400]	train-rmse:0.01493	valid-rmse:0.01082
[485]	train-rmse:0.01487	valid-rmse:0.01077
[0]	train-rmse:0.03625	valid-rmse:0.03450
[100]	train-rmse:0.02361	valid-rmse:0.02168
[200]	train-rmse:0.01783	valid-rmse:0.01559
[300]	train-rmse:0.01512	valid-rmse:0.01271
[400]	train-rmse:0.01488	valid-rmse:0.01246
[410]	train-rmse:0.01488	valid-rmse:0.01246
[0]	train-rmse:0.03554	valid-rmse:0.03738
[100]	train-rmse:0.02309	valid-rmse:0.02473
[200]	train-rmse:0.01745	valid-rmse:0.01913
[300]	train-rmse:0.01484	valid-rmse:0.01

In [ ]:
# Visualize feature importance
feature_names = train_features.drop('geology_id', axis=1).columns.tolist()
visualize_feature_importance(models, feature_names)